In [2]:
import pandas as pd
import numpy as np
import aux.acessos as ac
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [3]:
pb = pd.read_csv('outoftime_proba_decils_final.csv', sep =';')
len(pb)

134504

In [3]:
pb.head()

,Unnamed: 0,conta_cartao,proba,decil_score,year,month
0,0,4270104,0.121787,3,2022,5
1,1,4305804,0.119593,3,2022,5
2,2,4319104,0.162291,2,2022,5
3,3,44311750,0.049459,6,2022,5
4,4,44318360,0.141022,2,2022,5


In [4]:
pb_s1 = pb.iloc[0:50000,]

In [5]:
pb_s2 = pb.iloc[50001:80000,]

In [14]:
pb_s1['bin_temp'] = pd.qcut(pb_s1['proba'], q=10, precision=0, labels=False) 
tres = pb_s1.groupby('bin_temp').agg({'proba':['min', 'max']})
tres.columns = tres.columns.droplevel(0)
tres = tres.reset_index()
tres.head(10)

/tmp/ipykernel_499294/2682672314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pb_s1['bin_temp'] = pd.qcut(pb_s1['proba'], q=10, precision=0, labels=False)


,bin_temp,min,max
0,0,0.001053,0.003663
1,1,0.003663,0.008422
2,2,0.008426,0.025321
3,3,0.025322,0.044476
4,4,0.044491,0.063179
5,5,0.063181,0.081897
6,6,0.081901,0.104648
7,7,0.104655,0.138093
8,8,0.138097,0.212575
9,9,0.212654,0.928120


In [40]:
#df 1 - In sample - Score de treinamento
#df 2 - Out of time ou Teste

def psi(df1, score1, contagem1, df2, score2, contagem2):
    
    pd.set_option('mode.chained_assignment', None)
    df1['bin_score_df1'] = pd.qcut(df1[score1], q=10, precision=0, labels = False)
    tresh = df1.groupby('bin_score_df1').agg({score1:['min', 'max']})
    tresh.columns = tresh.columns.droplevel(0)
    tresh = tresh.reset_index()
        
    df2['bin_score_df2'] = np.where(df2[score2] <= tresh.iloc[0, 2], 0, 100)
    
    for i in range(1, 9):
        df2['bin_score_df2'] = np.where(df2['bin_score_df2'] != 100, df2['bin_score_df2'], 
                               np.where(df2[score2] <= tresh.iloc[i, 2], i, 100))
    
    agg1 = df1.groupby('bin_score_df1').agg({contagem1:['count']})
    agg1.columns = agg1.columns.droplevel(0)
    agg1 = agg1.reset_index()
    agg2 = df2.groupby('bin_score_df2').agg({contagem2:['count']})
    agg2.columns = agg2.columns.droplevel(0)
    agg2 = agg2.reset_index()
        
    agg3 = pd.DataFrame()
    agg3['dist1'] = agg1['count'] / (agg1['count'].sum())
    agg3['dist2'] = agg2['count'] / (agg2['count'].sum())
    agg3['psi'] = (agg3['dist1'] - agg3['dist2']) * np.log(agg3['dist1'] / agg3['dist2'])
    
   
    psi_score = sum(agg3['psi'])
    
    print("Conclusão:")
    if psi_score < 0.1:
        print("Sem alteração entre os scores")
    elif psi_score >= 0.1 and psi_score <= 0.25:
        print("Mudanças pouco significativas")
    elif psi_score > 0.25:
        print("Mudanças significativas")
    
    print("Valor do índice de estabilidade: ")
    return round(psi_score, 6)



In [41]:
psi(pb_s1, 'proba', 'conta_cartao', pb_s2, 'proba', 'conta_cartao')

Conclusão:
Sem alteração entre os scores
Valor do índice de estabilidade: 


0.000308